 <picture>
 <source media="(prefers-color-scheme: dark)" srcset="_images/astroquery_project_logo_dark.svg">
 <source media="(prefers-color-scheme: light)" srcset="_images/astroquery_project_logo.svg">
 <img  class= "dark-light" src="images/wasi-splash.svg" alt="WASI" style="float:right;height:60px;padding-right:20px;">
</picture>


# Astroquery Examples

Astroquery ( [https://astroquery.readthedocs.io](https://astroquery.readthedocs.io) )
is a set of tools for querying astronomical web forms and databases.

It is a coordinated package
of Astropy project ( [http://www.astropy.org](http://www.astropy.org/) ).

General imports:

In [1]:
from astropy import coordinates
import astropy.units as u

from astroquery.simbad import Simbad
from astroquery.vizier import Vizier

SIMBAD allows for simple queries based on coordinates or object names.

In [2]:
result_table = Simbad.query_object("M57")
result_table['MAIN_ID', 'RA', 'DEC'].show_in_notebook()

idx,MAIN_ID,RA,DEC
,,"""h:m:s""","""d:m:s"""
0,M 57,18 53 35.0967,+33 01 44.883


Wildcards and ranges can be used for identifiers

In [3]:
result_table = Simbad.query_object("m [1-9]", wildcard=True)
result_table['MAIN_ID', 'RA', 'DEC'].show_in_notebook()

idx,MAIN_ID,RA,DEC
,,"""h:m:s""","""d:m:s"""
0,M 1,05 34 30.9,+22 00 53
1,M 2,21 33 27.02,-00 49 23.7
2,M 3,13 42 11.62,+28 22 38.2
3,M 4,16 23 35.22,-26 31 32.7
4,M 5,15 18 33.22,+02 04 51.7
5,NGC 6405,17 40 16.6,-32 14 31
6,NGC 6475,17 53 47.3,-34 50 28
7,M 8,18 03 37,-24 23.2
8,M 9,17 19 11.78,-18 30 58.5


In [4]:
coord = coordinates.SkyCoord("18h53m35s +33d01m45s", frame='icrs')
radius = 5 * u.arcminute

result_table = Simbad.query_region(coord, radius=radius)
result_table['MAIN_ID', 'RA', 'DEC'].show_in_notebook()

idx,MAIN_ID,RA,DEC
,,"""h:m:s""","""d:m:s"""
0,M 57,18 53 35.0967,+33 01 44.883
1,UCAC3 247-143405,18 53 34.1995,+33 02 55.423
2,UCAC3 247-143401,18 53 33.8083,+33 02 54.923
3,UCAC2 43471160,18 53 34.2870,+33 03 14.297
4,GB6 J1853+3301,18 53 41,+33 03.0
5,1RXS J185326.3+330027,18 53 26.302,+33 00 27.00
6,LEDA 2813726,18 53 46.0,+32 58 53
7,IC 1296,18 53 18.8159,+33 03 59.894
